In [1]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime

In [3]:
import os
os.getcwd()

'C:\\Users\\sudhe\\Desktop\\Stock Market Research'

In [4]:
%load_ext tensorboard

In [5]:
def build_model():
    model=tf.keras.Sequential()

    model.add(tf.keras.layers.InputLayer(input_shape=(4,)))
    model.add(tf.keras.layers.Dense(25,activation='relu'))
    model.add(tf.keras.layers.Dense(15,activation='relu'))
    model.add(tf.keras.layers.Dense(5,activation='relu'))
    model.add(tf.keras.layers.Dense(1))

    optimizer = tf.keras.optimizers.Adam(0.001)

    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])                         
    return model

model=build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 25)                125       
_________________________________________________________________
dense_1 (Dense)              (None, 15)                390       
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 80        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 601
Trainable params: 601
Non-trainable params: 0
_________________________________________________________________


In [6]:
X=np.arange(0,2000).reshape(-1,4)
y=np.arange(5,(X.shape[0]+1)*5,5)

temp=pd.DataFrame(data=X)
temp['target']=pd.Series(y.reshape(-1,))
temp.head()

,0,1,2,3,target
0,0,1,2,3,5
1,4,5,6,7,10
2,8,9,10,11,15
3,12,13,14,15,20
4,16,17,18,19,25


In [7]:
from sklearn.preprocessing import StandardScaler
inputs_standardizer=StandardScaler()
inputs_standardizer.fit_transform(temp.drop('target',axis=1).values.tolist())

target_standardizer=StandardScaler()
target_standardizer.fit_transform(temp['target'].values.reshape(-1,1).tolist())

norm_temp=pd.DataFrame(data=inputs_standardizer.fit_transform(temp.drop('target',axis=1).values.tolist()))
norm_temp['target']=pd.Series(target_standardizer.fit_transform(temp['target'].values.reshape(-1,1).tolist()).reshape(-1,))
norm_temp.head()

,0,1,2,3,target
0,-1.728590,-1.728590,-1.728590,-1.728590,-1.728590
1,-1.721662,-1.721662,-1.721662,-1.721662,-1.721662
2,-1.714734,-1.714734,-1.714734,-1.714734,-1.714734
3,-1.707806,-1.707806,-1.707806,-1.707806,-1.707806
4,-1.700877,-1.700877,-1.700877,-1.700877,-1.700877


In [8]:
train_dataset = temp.sample(frac=0.8,random_state=0)
test_dataset = temp.drop(train_dataset.index)

In [9]:
train_stats = temp.describe()
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
0,500.0,998.0,577.927331,0.0,499.00,998.0,1497.00,1996.0
1,500.0,999.0,577.927331,1.0,500.00,999.0,1498.00,1997.0
2,500.0,1000.0,577.927331,2.0,501.00,1000.0,1499.00,1998.0
3,500.0,1001.0,577.927331,3.0,502.00,1001.0,1500.00,1999.0
target,500.0,1252.5,722.409164,5.0,628.75,1252.5,1876.25,2500.0


In [10]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)
normed_train_target=normed_train_data.pop('target')
normed_test_target=normed_test_data.pop('target')

In [11]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [11]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

model.fit(normed_train_data,normed_train_target,validation_split=0.2,epochs=240,verbose=1,batch_size=10,callbacks=[tensorboard_callback])

Epoch 1/240
 1/32 [..............................] - ETA: 0s - loss: 1.0224 - mae: 0.7670 - mse: 1.0224WARNING:tensorflow:From C:\Users\sudhe\Anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
32/32 [==============================] - 1s 30ms/step - loss: 0.8075 - mae: 0.7699 - mse: 0.8075 - val_loss: 0.6079 - val_mae: 0.6642 - val_mse: 0.6079
Epoch 2/240
32/32 [==============================] - 0s 10ms/step - loss: 0.4356 - mae: 0.5604 - mse: 0.4356 - val_loss: 0.2573 - val_mae: 0.4190 - val_mse: 0.2573
Epoch 3/240
32/32 [==============================] - 0s 6ms/step - loss: 0.1248 - mae: 0.2741 - mse: 0.1248 - val_loss: 0.0310 - val_mae: 0.1293 - val_mse: 0.0310
Epoch 4/240
32/32 [==============================] - 0s 6ms/step - loss: 0.0114 - mae: 0.0758 - mse: 0.0114 - val_loss: 0.0025 - val_mae

Epoch 43/240
32/32 [==============================] - 0s 5ms/step - loss: 2.5634e-06 - mae: 0.0012 - mse: 2.5634e-06 - val_loss: 2.8506e-06 - val_mae: 0.0012 - val_mse: 2.8506e-06
Epoch 44/240
32/32 [==============================] - 0s 4ms/step - loss: 2.2825e-06 - mae: 0.0011 - mse: 2.2825e-06 - val_loss: 2.4683e-06 - val_mae: 0.0011 - val_mse: 2.4683e-06
Epoch 45/240
32/32 [==============================] - 0s 4ms/step - loss: 2.4381e-06 - mae: 0.0011 - mse: 2.4381e-06 - val_loss: 2.4545e-06 - val_mae: 0.0011 - val_mse: 2.4545e-06
Epoch 46/240
32/32 [==============================] - 0s 4ms/step - loss: 2.1174e-06 - mae: 0.0010 - mse: 2.1174e-06 - val_loss: 2.3352e-06 - val_mae: 0.0010 - val_mse: 2.3352e-06
Epoch 47/240
32/32 [==============================] - 0s 4ms/step - loss: 2.1968e-06 - mae: 0.0011 - mse: 2.1968e-06 - val_loss: 2.2885e-06 - val_mae: 0.0011 - val_mse: 2.2885e-06
Epoch 48/240
32/32 [==============================] - 0s 4ms/step - loss: 2.0030e-06 - mae: 0.0010 -

32/32 [==============================] - 0s 4ms/step - loss: 7.4223e-07 - mae: 6.0502e-04 - mse: 7.4223e-07 - val_loss: 7.6285e-07 - val_mae: 6.3687e-04 - val_mse: 7.6285e-07
Epoch 88/240
32/32 [==============================] - 0s 8ms/step - loss: 7.1959e-07 - mae: 6.0081e-04 - mse: 7.1959e-07 - val_loss: 5.7667e-07 - val_mae: 4.6870e-04 - val_mse: 5.7667e-07
Epoch 89/240
32/32 [==============================] - 0s 4ms/step - loss: 7.0470e-07 - mae: 5.7142e-04 - mse: 7.0470e-07 - val_loss: 8.6436e-07 - val_mae: 6.4438e-04 - val_mse: 8.6436e-07
Epoch 90/240
32/32 [==============================] - 0s 4ms/step - loss: 7.5917e-07 - mae: 6.1387e-04 - mse: 7.5917e-07 - val_loss: 5.6573e-07 - val_mae: 4.8640e-04 - val_mse: 5.6573e-07
Epoch 91/240
32/32 [==============================] - 0s 5ms/step - loss: 6.9169e-07 - mae: 5.7893e-04 - mse: 6.9169e-07 - val_loss: 5.8166e-07 - val_mae: 4.9064e-04 - val_mse: 5.8166e-07
Epoch 92/240
32/32 [==============================] - 0s 4ms/step - loss:

32/32 [==============================] - 0s 5ms/step - loss: 2.8826e-07 - mae: 3.2227e-04 - mse: 2.8826e-07 - val_loss: 2.9890e-07 - val_mae: 3.3038e-04 - val_mse: 2.9890e-07
Epoch 131/240
32/32 [==============================] - 0s 4ms/step - loss: 4.4784e-07 - mae: 4.6874e-04 - mse: 4.4784e-07 - val_loss: 7.6288e-07 - val_mae: 7.5380e-04 - val_mse: 7.6288e-07
Epoch 132/240
32/32 [==============================] - 0s 4ms/step - loss: 7.1430e-07 - mae: 6.2908e-04 - mse: 7.1430e-07 - val_loss: 7.2546e-07 - val_mae: 7.3392e-04 - val_mse: 7.2546e-07
Epoch 133/240
32/32 [==============================] - 0s 5ms/step - loss: 7.3614e-07 - mae: 5.8789e-04 - mse: 7.3614e-07 - val_loss: 3.1491e-06 - val_mae: 0.0015 - val_mse: 3.1491e-06
Epoch 134/240
32/32 [==============================] - 0s 4ms/step - loss: 8.7555e-07 - mae: 6.4412e-04 - mse: 8.7555e-07 - val_loss: 2.9276e-07 - val_mae: 3.7062e-04 - val_mse: 2.9276e-07
Epoch 135/240
32/32 [==============================] - 0s 4ms/step - loss

Epoch 174/240
32/32 [==============================] - 0s 4ms/step - loss: 3.4542e-07 - mae: 4.3435e-04 - mse: 3.4542e-07 - val_loss: 6.0772e-07 - val_mae: 6.4148e-04 - val_mse: 6.0772e-07
Epoch 175/240
32/32 [==============================] - 0s 4ms/step - loss: 3.0811e-06 - mae: 0.0011 - mse: 3.0811e-06 - val_loss: 4.8041e-06 - val_mae: 0.0016 - val_mse: 4.8041e-06
Epoch 176/240
32/32 [==============================] - 0s 4ms/step - loss: 2.5491e-05 - mae: 0.0035 - mse: 2.5491e-05 - val_loss: 1.2093e-04 - val_mae: 0.0072 - val_mse: 1.2093e-04
Epoch 177/240
32/32 [==============================] - 0s 5ms/step - loss: 2.1902e-04 - mae: 0.0095 - mse: 2.1902e-04 - val_loss: 9.7552e-05 - val_mae: 0.0080 - val_mse: 9.7552e-05
Epoch 178/240
32/32 [==============================] - 0s 4ms/step - loss: 8.3096e-05 - mae: 0.0074 - mse: 8.3096e-05 - val_loss: 7.0132e-05 - val_mae: 0.0059 - val_mse: 7.0132e-05
Epoch 179/240
32/32 [==============================] - 0s 5ms/step - loss: 2.9924e-05 -

32/32 [==============================] - 0s 4ms/step - loss: 1.2116e-05 - mae: 0.0025 - mse: 1.2116e-05 - val_loss: 2.3273e-05 - val_mae: 0.0041 - val_mse: 2.3273e-05
Epoch 219/240
32/32 [==============================] - 0s 5ms/step - loss: 1.2762e-05 - mae: 0.0027 - mse: 1.2762e-05 - val_loss: 1.6940e-05 - val_mae: 0.0030 - val_mse: 1.6940e-05
Epoch 220/240
32/32 [==============================] - 0s 4ms/step - loss: 3.1077e-05 - mae: 0.0034 - mse: 3.1077e-05 - val_loss: 1.1383e-04 - val_mae: 0.0083 - val_mse: 1.1383e-04
Epoch 221/240
32/32 [==============================] - 0s 5ms/step - loss: 4.3818e-05 - mae: 0.0043 - mse: 4.3818e-05 - val_loss: 2.5664e-06 - val_mae: 0.0011 - val_mse: 2.5664e-06
Epoch 222/240
32/32 [==============================] - 0s 4ms/step - loss: 4.3741e-06 - mae: 0.0014 - mse: 4.3741e-06 - val_loss: 7.4100e-07 - val_mae: 6.8036e-04 - val_mse: 7.4100e-07
Epoch 223/240
32/32 [==============================] - 0s 5ms/step - loss: 1.0065e-06 - mae: 7.6981e-04 -

In [12]:
%tensorboard --logdir=logs/scalars/

Reusing TensorBoard on port 6006 (pid 7936), started 0:11:11 ago. (Use '!kill 7936' to kill it.)

In [13]:
%tensorboard --logdir=logs/scalars/'20200813-061118'

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 11456.

In [ ]:
y